# Métrica 1.1.10: "Congruencia del CURP a partir del nombre y la fecha de nacimiento (pareja)"

In [2]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
import traceback
# Asumimos que tienes un archivo config.py con tus variables de conexión
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_10_CURP_PAREJA_VS_NOMBRE"
SOURCE_COLLECTION_NAME = "all_data_20251021"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

def normalize_string_for_curp(name: str) -> str:
    """
    Limpia y normaliza un nombre/apellido para las reglas de CURP.
    - Quita acentos (ej. JOSÉ -> JOSE)
    - Convierte a mayúsculas
    - Maneja 'Ñ' -> 'X'
    - Quita prefijos y artículos comunes (DE, LA, DEL, Y, ETC.)
    - Toma el primer nombre/apellido en caso de compuestos.
    """
    if not isinstance(name, str):
        return ""
        
    # 1. Convertir a mayúsculas
    s = name.upper()
    
    # 2. Quitar acentos (ej. JOSÉ -> JOSE)
    s = ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn')
                
    # 3. Reemplazar 'Ñ'
    s = s.replace('Ñ', 'X')
    
    # 4. Quitar prefijos comunes (artículos, preposiciones)
    # Esta lista puede crecer
    prefixes_to_remove = [
        'DE ', 'DEL ', 'LA ', 'LOS ', 'LAS ', 'Y ', 'MC ', 'MAC ', 'VON ', 'VAN '
    ]
    
    for prefix in prefixes_to_remove:
        if s.startswith(prefix):
            s = s[len(prefix):]
            
    # 5. Tomar solo la primera palabra (manejo básico de nombres/apellidos compuestos)
    s = s.split(' ')[0]
    
    # 6. Quitar cualquier caracter no alfabético restante
    s = re.sub(r'[^A-Z]', '', s)
    
    return s

def get_internal_vowel(s: str) -> str:
    """Obtiene la primera vocal INTERNA (después del primer caracter)."""
    if len(s) <= 1:
        return 'X' # No tiene caracteres internos
    vowels = re.sub(r'[^AEIOU]', '', s[1:]) # Obtiene vocales de s[1] en adelante
    return vowels[0] if vowels else 'X' # Devuelve la primera o 'X'

def validar_curp_vs_nombre_fecha(curp: str, nombre: str, ap_paterno: str, ap_materno: str, fecha_nacimiento: str) -> str:
    """
    Valida la congruencia del CURP vs nombre y fecha de nacimiento
    según las reglas estándar (Métrica 1.1.10).
    """
    
    # --- Validación de Entradas ---
    # Necesitamos sí o sí CURP (18), nombre, primer apellido y fecha (10).
    if not all([curp, nombre, ap_paterno, fecha_nacimiento]):
        return "SIN_DATO"
        
    if len(curp) != 18 or len(fecha_nacimiento) != 10:
        # El CURP o la fecha no tienen la longitud correcta para ser analizados
        # (Fecha debe ser YYYY-MM-DD)
        return "SIN_DATO" 
        
    # --- Normalización de Nombres ---
    n_nombre = normalize_string_for_curp(nombre)
    n_paterno = normalize_string_for_curp(ap_paterno)
    n_materno = normalize_string_for_curp(ap_materno if ap_materno else "")
    
    # Si después de normalizar quedan vacíos, no podemos procesar
    if not all([n_nombre, n_paterno]):
        return "SIN_DATO"
        
    # --- Extracción de caracteres de la persona ---
    try:
        c_1_calc = n_paterno[0]
        c_2_calc = get_internal_vowel(n_paterno)
        c_3_calc = n_materno[0] if n_materno else 'X'
        c_4_calc = n_nombre[0]
        
        # --- Extracción de Fecha ---
        year_short = fecha_nacimiento[2:4]
        month = fecha_nacimiento[5:7]
        day = fecha_nacimiento[8:10]
        c_5_10_calc = f"{year_short}{month}{day}"
        
    except IndexError:
        # Esto pasaría si un nombre normalizado es una sola letra (ej. "A")
        return "NO_CUMPLE"
        
    # --- Extracción de caracteres del CURP ---
    c_1_10_curp = curp[0:10]
    
    # --- Construcción del CURP calculado ---
    c_1_10_calc = f"{c_1_calc}{c_2_calc}{c_3_calc}{c_4_calc}{c_5_10_calc}"
    
    # --- Comparación Final ---
    if c_1_10_calc == c_1_10_curp:
        return "CUMPLE"
    else:
        return "NO_CUMPLE"

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_curp_pareja_full():
    """
    Procesa TODAS las declaraciones para la Métrica 1.1.10
    """
    client = None
    # Contadores
    total_documentos = 0
    cumple = 0
    no_cumple = 0
    sin_dato = 0
    na = 0 # No Aplica (para quienes no tienen pareja)
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"\nConectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"--- INICIANDO PROCESAMIENTO (Métrica 1.1.10 - CURP vs Nombre/Fecha Pareja) ---")

        # Proyección optimizada para traer el 'id' y la sección 'datosPareja'
        proyeccion = {
            "_id": 1,
            "id": 1,
            "declaracion.situacionPatrimonial.datosPareja": 1 
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]
            string_id = doc.get("id", "N/D")
            
            curp = None
            nombre = None
            ap_paterno = None
            ap_materno = None
            fecha_nacimiento = None
            
            resultado_metrica = "SIN_DATO" # Valor por defecto
            
            try:
                # Navegación segura
                datos_pareja = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosPareja")

                # --- Lógica de la Métrica ---
                if not isinstance(datos_pareja, dict):
                    resultado_metrica = "SIN_DATO"
                
                elif datos_pareja.get("ninguno") is True:
                    resultado_metrica = "N/A"
                
                else:
                    # El declarante SÍ tiene pareja, buscamos todos los campos
                    curp = datos_pareja.get("curp")
                    nombre = datos_pareja.get("nombre")
                    ap_paterno = datos_pareja.get("primerApellido")
                    ap_materno = datos_pareja.get("segundoApellido")
                    fecha_nacimiento = datos_pareja.get("fechaNacimiento")
                    
                    # Ejecutamos la validación (la función se encarga del SIN_DATO si falta algo)
                    resultado_metrica = validar_curp_vs_nombre_fecha(curp, nombre, ap_paterno, ap_materno, fecha_nacimiento)

            except AttributeError:
                resultado_metrica = "SIN_DATO" # Ocurrió un error leyendo la estructura
            
            # Actualizamos contadores
            if resultado_metrica == "CUMPLE": cumple += 1
            elif resultado_metrica == "NO_CUMPLE": no_cumple += 1
            elif resultado_metrica == "N/A": na += 1
            else: sin_dato += 1 # SIN_DATO

            # --- CONSERVAMOS TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id} (id: {string_id})")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > CURP Pareja: '{curp}'")
            print(f"  > Nombre: '{nombre}', Paterno: '{ap_paterno}', Materno: '{ap_materno}'")
            print(f"  > Fecha Nac: '{fecha_nacimiento}'")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")

            # --- E. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            filtro = { "_id": original_id }
            actualizacion = { "$set": { METRIC_ID: resultado_metrica } }
            target_collection.update_one(filtro, actualizacion, upsert=True)
            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.10):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {no_cumple}")
        print(f"  > Métrica 'N/A' (Sin pareja declarada): {na}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        traceback.print_exc()
    finally:
        if client: client.close(); print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones_curp_pareja_full()


Conectando a MongoDB en sistema1...
¡Conexión exitosa!
--- INICIANDO PROCESAMIENTO (Métrica 1.1.10 - CURP vs Nombre/Fecha Pareja) ---

Procesando Documento ID: 68f81b8800535f910a29f694 (id: 194916)
--- RESULTADO DEL ANÁLISIS ---
  > CURP Pareja: 'None'
  > Nombre: 'None', Paterno: 'None', Materno: 'None'
  > Fecha Nac: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695 (id: 59837)
--- RESULTADO DEL ANÁLISIS ---
  > CURP Pareja: 'None'
  > Nombre: 'None', Paterno: 'None', Materno: 'None'
  > Fecha Nac: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696 (id: 31074)
--- RESULTADO DEL ANÁLISIS ---
  > CURP Pareja: 'None'
  > Nombre: 'None', Paterno: 'None', Materno: 'N

KeyboardInterrupt: 